In [2]:
#currently using the small extracted datafile with 24 testdata; 
# keep getting error on loss=criterion(outputs,y)---- target out of bound
import torch
import torch.nn as nn
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device="cpu"
torch.cuda.empty_cache() 

class Block(nn.Module):
    def __init__(self, num_layers, in_channels, out_channels, identity_downsample=None, stride=1):
        assert num_layers in [18, 34, 50, 101, 152], "should be a a valid architecture"
        super(Block, self).__init__()
        self.num_layers = num_layers
        if self.num_layers > 34:
            self.expansion = 4
        else:
            self.expansion = 1
        # ResNet50, 101, and 152 include additional layer of 1x1 kernels
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        if self.num_layers > 34:
            self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        else:
            # for ResNet18 and 34, connect input directly to (3x3) kernel (skip first (1x1))
            self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x
        if self.num_layers > 34:
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, num_layers, block, image_channels, num_classes):
        assert num_layers in [18, 34, 50, 101, 152], f'ResNet{num_layers}: Unknown architecture! Number of layers has ' \
                                                     f'to be 18, 34, 50, 101, or 152 '
        super(ResNet, self).__init__()
        if num_layers < 50:
            self.expansion = 1
        else:
            self.expansion = 4
        if num_layers == 18:
            layers = [2, 2, 2, 2]
        elif num_layers == 34 or num_layers == 50:
            layers = [3, 4, 6, 3]
        elif num_layers == 101:
            layers = [3, 4, 23, 3]
        else:
            layers = [3, 8, 36, 3]
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNetLayers
        self.layer1 = self.make_layers(num_layers, block, layers[0], intermediate_channels=64, stride=1)
        self.layer2 = self.make_layers(num_layers, block, layers[1], intermediate_channels=128, stride=2)
        self.layer3 = self.make_layers(num_layers, block, layers[2], intermediate_channels=256, stride=2)
        self.layer4 = self.make_layers(num_layers, block, layers[3], intermediate_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * self.expansion, num_classes)
        

        
        
    def forward(self, x):
        print("forward "+str(x.size()))
        x = self.conv1(x)
        print("forward after conv1 "+str(x.size()))
        x = self.bn1(x)
        print("forward after bn1 "+str(x.size()))
        x = self.relu(x)
        print("forward after relu "+str(x.size()))
        x = self.maxpool(x)
        print("forward after maxpool "+str(x.size()))
        x = self.layer1(x)
        print("forward "+str(x.size()))
        x = self.layer2(x)
        print("forward "+str(x.size()))
        x = self.layer3(x)
        print("forward "+str(x.size()))
        x = self.layer4(x)
        print("forward "+str(x.size()))
        x = self.avgpool(x)
        print("forward "+str(x.size()))
        x = x.reshape(x.shape[0], -1)
        print("forward "+str(x.size()))
        x = self.fc(x)
        print("forward "+str(x.size()))
        return x

    def make_layers(self, num_layers, block, num_residual_blocks, intermediate_channels, stride):
        layers = []

        identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, intermediate_channels*self.expansion, kernel_size=1, stride=stride),
                                            nn.BatchNorm2d(intermediate_channels*self.expansion))
        layers.append(block(num_layers, self.in_channels, intermediate_channels, identity_downsample, stride))
        self.in_channels = intermediate_channels * self.expansion # 256
        for i in range(num_residual_blocks - 1):
            layers.append(block(num_layers, self.in_channels, intermediate_channels)) # 256 -> 64, 64*4 (256) again
        return nn.Sequential(*layers)


def ResNet18(img_channels=3, num_classes=1000):
    return ResNet(18, Block, img_channels, num_classes)


def ResNet34(img_channels=3, num_classes=1000):
    return ResNet(34, Block, img_channels, num_classes)


def ResNet50(img_channels=3, num_classes=1000):
    return ResNet(50, Block, img_channels, num_classes)


def ResNet101(img_channels=3, num_classes=1000):
    return ResNet(101, Block, img_channels, num_classes)


def ResNet152(img_channels=3, num_classes=1000):
    return ResNet(152, Block, img_channels, num_classes)


class MyDataSet(Dataset):
        def __init__(self,X,Y):
            self.X=X
            self.Y=Y
            self.N=self.X.shape[0]
            self.K=self.X.shape[1]
        def __len__(self):
            return(self.N)
        def __getitem__(self, idx):
            return self.X[idx],self.Y[idx]

Xtrain=np.load("MusicTrainingImagesReshaped2.npy")
print(Xtrain.shape)
print(Xtrain[1][0])

(3, 1, 513, 245)
[[0.85420213 0.85244615 0.78359503 ... 0.84777877 0.84317175 0.84454527]
 [0.98867201 0.93291589 0.93872781 ... 0.9161695  0.92045293 0.9191017 ]
 [0.94581499 0.92631355 0.93578885 ... 0.95866542 0.955151   0.9596596 ]
 ...
 [0.35289731 0.41147769 0.40065432 ... 0.39905583 0.38606621 0.36785466]
 [0.37948273 0.41369067 0.37228786 ... 0.39083571 0.37346232 0.39343748]
 [0.40716169 0.31949731 0.31204434 ... 0.35099331 0.29015256 0.39404925]]


In [4]:
Ytrain=np.load("MusicTrainingLabels2.npy")

Xtest=np.load("MusicTestingImagesReshaped2.npy")
Ytest=np.load("MusicTestingLabels2.npy")
ntest=Xtest.shape[0]
print("data read from disk")

TrainingData=MyDataSet(Xtrain,Ytrain)
TestingData=MyDataSet(Xtest,Ytest)

print("created training & testing data")
print(type(Ytrain))
print(Ytrain[0])

data read from disk
created training & testing data
<class 'numpy.ndarray'>
3


In [11]:
DataLoader_train=torch.utils.data.DataLoader(
    dataset=TrainingData,
    batch_size=3,
    shuffle=True)

print("created data loader")

learning_rate = .01
num_epochs=1

model = ResNet18(img_channels=1, num_classes=2)

print("created model")


#
# move the model to the device
#
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
#
# Adam is an optimizer that maintains a different learning 
# rate for every weight/parameter in the network - so the learning rate 
# is a relative one
#
# See the gentle introduction:
#        e.g. https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/
#
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    print("epoch = "+str(epoch))
    #
    # for each epoch the data loader loads the data one batch at a time
    #
    for i,(x,y) in enumerate(DataLoader_train):
        if i>1:
            break
        print(x.shape)
        print("i = "+str(i))
        x=x.float()
        x=x.to(device)
        y=y.to(device,dtype=torch.long)
        print(y,len(y),'y')

        optimizer.zero_grad()

        print(x.size(),'x size')
        outputs=model(x)
        print(len(outputs), 'output size')

        loss=criterion(outputs,y)
        loss_value = loss.item()

        loss.backward() # compute gradients

        # move in direction of minus the gradient by a learning_rate amount 
        # here because we are using Adam, step is more complicated than -epsilon*Gradient
        optimizer.step() 
        #print("epoch = {0:5d} i = {1:5d} loss = {2:8.5f}".format(epoch,i,loss_value))
    print("epoch = {0:5d} loss = {1:8.5f}".format(epoch,loss_value))
#
nclasses=2
Confusion=np.zeros(shape=(nclasses,nclasses),dtype=int)
xtest=torch.tensor(Xtest).float().to(device)
ypred=model(xtest)
ypred=ypred.cpu().detach().numpy()
ypred=np.apply_along_axis(np.argmax,1,ypred)
s=0
for i in range(4):
    Confusion[Ytest[i],ypred[i]]+=1
    s+=(Ytest[i]==ypred[i])
accuracy=s/4

created data loader
created model
epoch = 0
torch.Size([3, 1, 513, 245])
i = 0
tensor([3, 2, 0]) 3 y
torch.Size([3, 1, 513, 245]) x size
forward torch.Size([3, 1, 513, 245])
forward after conv1 torch.Size([3, 64, 257, 123])
forward after bn1 torch.Size([3, 64, 257, 123])
forward after relu torch.Size([3, 64, 257, 123])
forward after maxpool torch.Size([3, 64, 129, 62])
forward torch.Size([3, 64, 129, 62])
forward torch.Size([3, 128, 65, 31])
forward torch.Size([3, 256, 33, 16])
forward torch.Size([3, 512, 17, 8])
forward torch.Size([3, 512, 1, 1])
forward torch.Size([3, 512])
forward torch.Size([3, 2])
3 output size


IndexError: Target 3 is out of bounds.

In [ ]:
Confusion

In [ ]:
s

In [8]:
s/10000

0.0